In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import fiftyone
import math
import transformers
from utils.preprocessing import bbs_corners_to_centers, normalize_bbs 


import torchvision.datasets as datasets
import torchvision.transforms as transforms

import copy
import logging
import os
import pandas as pd
from torchvision.io import read_image

from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

In [5]:
# class ConvBlock(nn.Module):
#     def __init__(self, channels_out=64, kernel_size=3, stride=1, padding=1, bias=False):
#         super().__init__(ConvBlock, self)
#         self.conv = nn.LazyConv2d(channels_out, kernel_size, stride, padding, bias=False) 
#         self.pool = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
#         self.bn = nn.BatchNorm2d(channels_out)
#         self.act = nn.SiLU()
        
#     def forward(self, x):
#         return self.act(self.bn(self.pool(self.conv(x))))


In [6]:
class Residual(nn.Module):
    def __init__(self, num_channels, use1x1=False, strides=1):
        super().__init__()
        self.conv1 = nn.LazyConv2d(num_channels, kernel_size=3, padding=1, stride=strides)
        self.conv2 = nn.LazyConv2d(num_channels, kernel_size=3, padding=1)
        
        if use1x1:
            self.conv3 = nn.LazyConv2d(num_channels, kernel_size=1, strides=strides)
            
        else:
            self.conv3 = None
            
        self.bn1 = nn.LazyBatchNorm2d()
        self.bn2 = nn.LazyBatchNorm2d()
        
    def forward(self, x):
        y = F.SiLU(self.bn1(self.conv1(x)))
        y = self.bn2(self.conv2(y))
        if self.conv3:
            x = self.conv3(x)
        y += x
        return F.SiLU(y)

In [6]:
class BackBone(nn.Module):
    def __init__(self):
        super().__init__(BackBone, self)
        layer1 = nn.Sequential(
            nn.LazyConv2d(64, kernel_size=7, stride=2, padding=3), # Downsample 1/2
            nn.LazyBatchNorm2d(), nn.SiLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1) # Downsample 1/2
        )
        layer2 = Residual(64, True, 1)
        layer3 = Residual(32, True, 1)
        

In [3]:
class SelfAttention(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, q, k, v, temp=None):
        
        if temp == None:
          temp = k.size(-1)  # increasing temp will "flatten" the attention distribution 
        
        print(q.shape)
        
        att = q @ k.transpose(-2, -1) / math.sqrt(temp) # B, nh, T, T 
        att = F.softmax(att, dim=-1) 
        y = att @ v # B, nh, T, hs
        
        return y.transpose(1,2).contiguous()
        

In [3]:
class NewGELU(nn.Module):
    """
    Implementation of the GELU activation function currently in Google BERT repo (identical to OpenAI GPT).
    Reference: Gaussian Error Linear Units (GELU) paper: https://arxiv.org/abs/1606.08415
    """
    def forward(self, x):
        return 0.5 * x * (1.0 + torch.tanh(math.sqrt(2.0 / math.pi) * (x + 0.044715 * torch.pow(x, 3.0))))

In [4]:
class ConvBlock(nn.Module):
    def __init__(self, o_c, k_s, s=1, p=0, downsample=False):
        super().__init__()
        
        self.in1 = nn.LazyInstanceNorm2d()
        # self.in2 = nn.LazyInstanceNorm2d()
        
        self.conv1 = nn.LazyConv2d(o_c, k_s, s, p)
        self.pool = nn.MaxPool2d(2, 2) if downsample else nn.MaxPool2d(3, 1, 1)
        self.act = nn.ReLU()
        
    def forward(self, x):
        return self.act(self.pool(self.conv1(self.in1(x))))

In [5]:
class MultiheadSelfAttention(nn.Module):
    def __init__(self, emb_d, num_heads, dropout=0.0):
        super().__init__()
        
        assert emb_d % num_heads == 0
        
        self.emb_d = emb_d
        self.num_heads = num_heads
        self.c_attn = nn.Linear(emb_d, 3 * emb_d, bias=False)         
        self.attention = SelfAttention()
        self.conv = ConvBlock(emb_d, 3, 1, 1)
        
    def forward(self, x):
                    
        B, C, H, W = x.size()
        T = W * H
        x = x.view(B, C, T).transpose(1,2) # (B, T, C)
        
        q, k, v = self.c_attn(x).split(self.emb_d, dim=2)
        
        q = q.view(B, T, self.num_heads, C // self.num_heads).transpose(1, 2) # (B, nh, T, hs)
        k = k.view(B, T, self.num_heads, C // self.num_heads).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, T, self.num_heads, C // self.num_heads).transpose(1, 2) # (B, nh, T, hs)
        
        y = self.attention(q, k, v) # B, nh, hs, T
        
        y = y.view(B,T,C).transpose(1,2).view(B,C,H,W)
        out = self.conv(y)
                
        return out
        

        

In [6]:
x = torch.tensor([[[[1,2,3], [4,5,6]], [[7,8,9], [10,11,12]], [[13,14,15], [16,17,18]],[[19,20,21], [22,23,24]]]])
                # channel 1            # channel 2
                
nh = 2
                
B, C, H, W = x.size()

T = W * H

# x.view(B,C,T).transpose(1,2).view(B, T, nh, C // nh).transpose(1,2).contiguous().transpose(-2,-1).reshape(B,C,T).view(B,C,H,W)
x.view(B,C,T).transpose(1,2).view(B, T, nh, C // nh).transpose(1,2).transpose(1,2).contiguous().view(B,T,C).transpose(1,2).view(B,C,H,W)
# x.view(B,C,T).transpose(1,2).view(B, T, nh, C // nh).transpose(1,2).view(B, C, T)


tensor([[[[ 1,  2,  3],
          [ 4,  5,  6]],

         [[ 7,  8,  9],
          [10, 11, 12]],

         [[13, 14, 15],
          [16, 17, 18]],

         [[19, 20, 21],
          [22, 23, 24]]]])

In [49]:
class Net(nn.Module):
    def __init__(self,  num_classes:int):
        super(Net, self).__init__()
                
        self.conv1 = ConvBlock(512, 7, 1, 3)
        # self.mha1 = nn.MultiheadAttention(512, 8) 
        self.conv2 = ConvBlock(512, 5, 1, 2, downsample=True)
        # self.mha2 = nn.MultiheadAttention(512, 8)
        self.conv3 = ConvBlock(512, 3, 1, 1, downsample=True)
        
        self.bb_pred = nn.Sequential(
            ConvBlock(256, 3, 1, 1, downsample=True), ConvBlock(4, 1, 1, 0), ConvBlock(4, 3, 1, 0), nn.Flatten()
        ) # (B, 4, H/8, W/8)
        
        self.class_pred = nn.Sequential(
            ConvBlock(256, 3, 1, 1, downsample=True), ConvBlock(num_classes, 1, 1, 0), ConvBlock(num_classes, 1, 1), nn.Softmax(1) 
        ) # (B, num_classes, H/8, W/8)

        
    def forward(self, x):
        
        x = self.conv1(x)
        x = x.view(x.shape[0], 512, -1).transpose(-2, -1)
        x, _ = self.mha1(x, x, x)
        x = x.transpose(-2,-1).view(x.shape[0], 512, 128, 128)
        x = self.conv2(x)
        x = x.view(x.shape[0], 512, -1).transpose(-2, -1)
        x, _ = self.mha2(x, x, x)
        x = x.transpose(-2,-1).view(x.shape[0], 512, 64, 64)
        x = self.conv3(x)
                
        return self.bb_pred(x), self.class_pred(x)
    
        

In [8]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0,0,0), (1,1,1))]
)

train_dataset = datasets.VOCDetection('data\PASCALVOC', image_set='train', transform=transform, download=False)

val_dataset = datasets.VOCDetection('data\PASCALVOC', image_set='val', transform=transform, download=False)

# trainval_dataset = datasets.VOCDetection('data\PASCALVOC', image_set='trainval', transform=transform, download=True)


In [9]:
label_2_idx = {'pottedplant': 1, 'person': 2,
               'motorbike': 3, 'train': 4,
               'dog': 5, 'diningtable': 6,
               'horse': 7, 'bus': 8,
               'aeroplane': 9, 'sofa': 10,
               'sheep': 11, 'tvmonitor': 12,
               'bird': 13, 'bottle': 14,
               'chair': 15, 'cat': 16,
               'bicycle': 17, 'cow': 18,
               'boat': 19, 'car': 20, 'bg': 0}

idx_2_label = {1: 'pottedplant', 2: 'person',
               3: 'motorbike', 4: 'train',
               5: 'dog', 6: 'diningtable',
               7: 'horse', 8: 'bus',
               9: 'aeroplane', 10: 'sofa',
               11: 'sheep', 12: 'tvmonitor',
               13: 'bird', 14: 'bottle',
               15: 'chair', 16: 'cat',
               17: 'bicycle', 18: 'cow',
               19: 'boat', 20: 'car', 0: 'bg'}

In [31]:
class CustomVOCDetection(torch.utils.data.Dataset):
    def __init__(self, annotations:pd.DataFrame, img_dir, transform=None, target_transform=None):
        self.img_labels = annotations
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [ ]:
train_df = modify_dataset(train_dataset, label_2_idx)
val_df = modify_dataset(val_dataset, label_2_idx)

In [51]:
train_dataset = CustomVOCDetection(train_df, )

tensor([7, 2])

In [52]:
# TODO convert this into a "target_transform" function that'll be used by the Dataset object

def modify_dataset(dataset:torch.utils.data.Dataset, label_2_idx_map:dict) -> pd.DataFrame:

    try:                   

        new_labels = []
        
        for i in range(len(dataset)):

            annotation = dataset[i][1]['annotation']

            imgw = int(annotation['size']['width'])  
            imgh = int(annotation['size']['height'])         

            labels = annotation['object']

            bbs = []
            classes = []

            assert isinstance(labels, (list, dict)) 

            if isinstance(labels, list):
                
                for obj in labels:
                    # TODO replace innards with function call(s)
                    class_num = label_2_idx_map[obj['name']] if type(obj['name']) == str else label_2_idx_map[obj['name'][0]]

                    bb = obj['bndbox']
                    
                    assert type(bb) == dict

                    x1 = int(bb['xmin'][0]) if type(bb['xmin']) == list else int(bb['xmin'])
                    x2 = int(bb['xmax'][0]) if type(bb['xmax']) == list else int(bb['xmax'])
                    y1 = int(bb['ymin'][0]) if type(bb['ymin']) == list else int(bb['ymin'])
                    y2 = int(bb['ymax'][0]) if type(bb['ymax']) == list else int(bb['ymax'])

                    bb = [x1, y1, x2, y2]
                    bbs.append(bb)
                    classes.append(class_num)

            elif isinstance(labels, dict):
                # TODO replace innards with function call(s)
                obj = labels

                class_num = label_2_idx_map[obj['name']] if type(obj['name']) == str else label_2_idx_map[obj['name'][0]]

                assert type(bb) == dict

                bb = obj['bndbox']
                x1 = int(bb['xmin'][0]) if type(bb['xmin']) == list else int(bb['xmin'])
                x2 = int(bb['xmax'][0]) if type(bb['xmax']) == list else int(bb['xmax'])
                y1 = int(bb['ymin'][0]) if type(bb['ymin']) == list else int(bb['ymin'])
                y2 = int(bb['ymax'][0]) if type(bb['ymax']) == list else int(bb['ymax'])

                bb = [x1, y1, x2, y2]
                bbs.append(bb)

            bbs = torch.tensor(bbs)
            bbs = bbs_corners_to_centers(bbs)
            bbs = normalize_bbs(bbs, (imgw, imgh))
            
            new_label = (bbs, torch.tensor(classes))            
            new_labels.append(new_label)
                            
        return pd.DataFrame(new_labels)
    
    except AssertionError:
        print(f'Data entry {i} is not formatted as expected.\nNote that none of the dataset has been updated')
        return None
    
    except Exception as ex:
        logging.error(f'A general exception was caught: {ex}', exc_info=True)
        return None
        

In [81]:
type(str)

type

In [110]:
# Data loaders

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=True, pin_memory=True)

val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=True, pin_memory=True)

# trainval_loader = torch.utils.data.DataLoader(trainval_dataset, batch_size=1, shuffle=True)

# print('Trainval set has {} instances'.format(len(trainval_loader)))

print('Training set has {} instances'.format(len(train_loader)))
print('Validation set has {} instances'.format(len(val_loader)))

Training set has 5717 instances
Validation set has 5823 instances


In [165]:
for i in range(len(train_dataset)):
    
    if i == 5: break
        
    print(train_dataset[i][1]['annotation'])

{'folder': 'VOC2012', 'filename': '2008_000008.jpg', 'source': {'database': 'The VOC2008 Database', 'annotation': 'PASCAL VOC2008', 'image': 'flickr'}, 'size': {'width': '500', 'height': '442', 'depth': '3'}, 'segmented': '0', 'object': [{'name': 'horse', 'pose': 'Left', 'truncated': '0', 'occluded': '1', 'bndbox': {'xmin': '53', 'ymin': '87', 'xmax': '471', 'ymax': '420'}, 'difficult': '0'}, {'name': 'person', 'pose': 'Unspecified', 'truncated': '1', 'occluded': '0', 'bndbox': {'xmin': '158', 'ymin': '44', 'xmax': '289', 'ymax': '167'}, 'difficult': '0'}]}
{'folder': 'VOC2012', 'filename': '2008_000015.jpg', 'source': {'database': 'The VOC2008 Database', 'annotation': 'PASCAL VOC2008', 'image': 'flickr'}, 'size': {'width': '500', 'height': '327', 'depth': '3'}, 'segmented': '1', 'object': [{'name': 'bottle', 'pose': 'Unspecified', 'truncated': '1', 'occluded': '1', 'bndbox': {'xmin': '270', 'ymin': '1', 'xmax': '378', 'ymax': '176'}, 'difficult': '0'}, {'name': 'bottle', 'pose': 'Unsp

In [199]:
from utils.preprocessing import bbs_corners_to_centers, normalize_bbs

for i, data in enumerate(train_loader):
    if i == 1: break
    inputs, labels = data
    
    imgw = int(labels['annotation']['size']['width'][0])
    imgh = int(labels['annotation']['size']['height'][0])
    
    labels = labels['annotation']['object']
        
    if type(labels) == list:
    
        bbs = []

        for obj in labels:

            name = obj['name']

            bb = obj['bndbox']
            x1 = int(bb['xmin'][0]) if type(bb['xmin']) == list else bb['xmin']        
            x2 = int(bb['xmax'][0]) if type(bb['xmax']) == list else bb['xmax']
            y1 = int(bb['ymin'][0]) if type(bb['ymin']) == list else bb['ymin']
            y2 = int(bb['ymax'][0]) if type(bb['ymax']) == list else bb['ymax']

            bb = [x1, y1, x2, y2]
            bbs.append(bb)

        bb_lbl = torch.tensor(bbs)
    
    else:
        pass
    
    print(i)
    print(bbs)
    print('\n')
    print(bb_lbl)
    print('\n')
    print(bbs_corners_to_centers(bb_lbl))
    print('\n')
    print(f'img w: {imgw}, img h: {imgh}')
    print(normalize_bbs(bbs_corners_to_centers(bb_lbl), imgsz=(imgw, imgh)))

0
[[272, 111, 318, 155], [212, 131, 309, 216], [1, 1, 437, 333], [2, 30, 500, 333]]


tensor([[272, 111, 318, 155],
        [212, 131, 309, 216],
        [  1,   1, 437, 333],
        [  2,  30, 500, 333]])


tensor([[295.0000, 133.0000,  46.0000,  44.0000],
        [260.5000, 173.5000,  97.0000,  85.0000],
        [219.0000, 167.0000, 436.0000, 332.0000],
        [251.0000, 181.5000, 498.0000, 303.0000]])


img w: 500, img h: 333
tensor([[0.5900, 0.3994, 0.1381, 0.1321],
        [0.5210, 0.5210, 0.2913, 0.2553],
        [0.4380, 0.5015, 1.3093, 0.9970],
        [0.5020, 0.5450, 1.4955, 0.9099]])


In [132]:
train_dataset[3][1]['annotation']['object']

[{'name': 'tvmonitor',
  'pose': 'Frontal',
  'truncated': '1',
  'occluded': '1',
  'bndbox': {'xmin': '6', 'ymin': '1', 'xmax': '314', 'ymax': '262'},
  'difficult': '0'},
 {'name': 'bottle',
  'pose': 'Unspecified',
  'truncated': '1',
  'occluded': '1',
  'bndbox': {'xmin': '40', 'ymin': '97', 'xmax': '121', 'ymax': '411'},
  'difficult': '0'},
 {'name': 'person',
  'pose': 'Frontal',
  'truncated': '1',
  'occluded': '0',
  'bndbox': {'xmin': '137', 'ymin': '36', 'xmax': '169', 'ymax': '109'},
  'difficult': '0'},
 {'name': 'person',
  'pose': 'Frontal',
  'truncated': '1',
  'occluded': '0',
  'bndbox': {'xmin': '180', 'ymin': '36', 'xmax': '216', 'ymax': '104'},
  'difficult': '0'},
 {'name': 'person',
  'pose': 'Frontal',
  'truncated': '1',
  'occluded': '0',
  'bndbox': {'xmin': '96', 'ymin': '39', 'xmax': '123', 'ymax': '103'},
  'difficult': '0'}]

In [31]:
lin = nn.LazyLinear(10, bias=False)
X = torch.randn(1,1,5)
lin(X)
[p.numel() for p in lin.parameters()]

[50]

In [50]:
net = Net(num_classes=2).to('cuda')
opt = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [52]:
sum([p.numel() for p in net.parameters()])

11685810

In [53]:
X = torch.randn(5, 3, 128, 128).to('cuda')
net(X)


(tensor([[0.5570, 0.8553, 0.5165, 0.5202],
         [0.5570, 0.8553, 0.5164, 0.5202],
         [0.5570, 0.8553, 0.5164, 0.5202],
         [0.5570, 0.8553, 0.5164, 0.5202],
         [0.5570, 0.8553, 0.5165, 0.5202]], device='cuda:0',
        grad_fn=<ViewBackward0>),
 tensor([[[[0.4967, 0.4967, 0.4967,  ..., 0.4986, 0.5604, 0.6707],
           [0.4967, 0.3777, 0.3777,  ..., 0.3954, 0.6477, 0.7461],
           [0.5600, 0.4381, 0.4133,  ..., 0.3967, 0.6511, 0.7461],
           ...,
           [0.1526, 0.1526, 0.2365,  ..., 0.3529, 0.4746, 0.4746],
           [0.7494, 0.7494, 0.6167,  ..., 0.5186, 0.6356, 0.6356],
           [0.7677, 0.7677, 0.6167,  ..., 0.5186, 0.6356, 0.6356]],
 
          [[0.5033, 0.5033, 0.5033,  ..., 0.5014, 0.4396, 0.3293],
           [0.5033, 0.6223, 0.6223,  ..., 0.6046, 0.3523, 0.2539],
           [0.4400, 0.5619, 0.5867,  ..., 0.6033, 0.3489, 0.2539],
           ...,
           [0.8474, 0.8474, 0.7635,  ..., 0.6471, 0.5254, 0.5254],
           [0.2506, 0.2506, 

In [370]:
seq = nn.Sequential(ConvBlock(512, 7, 1, 3, downsample=True), ConvBlock(512, 3, 1, 1, downsample=True), ConvBlock(4, 3, 1, 1), nn.FractionalMaxPool2d((1,1), 1), nn.Flatten())

In [375]:
X = torch.rand(5,3,16,16)
Y = seq(X)

In [372]:
Y[:,:]

tensor([[0.7605, 0.1296, 1.0942, 0.2514],
        [0.5315, 1.1441, 0.4716, 0.3814],
        [0.3145, 0.2099, 0.6240, 0.7039],
        [0.0996, 0.7513, 0.4232, 1.0020],
        [0.2571, 0.2531, 0.5619, 0.6083]], grad_fn=<SliceBackward0>)

In [383]:
bb_labels = torch.empty(5, 4).uniform_(0, to=1)
bb_labels[:,:2]

tensor([[0.9524, 0.8184],
        [0.0786, 0.1661],
        [0.5139, 0.7119],
        [0.3975, 0.8346],
        [0.1803, 0.6426]])

In [398]:
bb_labels[:,:2].transpose(0,1)

tensor([[0.9524, 0.0786, 0.5139, 0.3975, 0.1803],
        [0.8184, 0.1661, 0.7119, 0.8346, 0.6426]])

In [396]:
Y[:, :2] @ bb_labels[:,:2].transpose(0,1)



tensor([[0.9768, 0.1982, 0.8496, 0.9961, 0.7670],
        [0.5050, 0.0417, 0.2725, 0.2108, 0.0956],
        [0.6510, 0.1230, 0.5413, 0.6182, 0.4661],
        [1.4457, 0.2095, 1.0274, 1.0545, 0.7199],
        [0.5524, 0.1020, 0.4527, 0.5126, 0.3835]], grad_fn=<MmBackward0>)

In [384]:
for y in Y:
    
    # print(f'y:\n{y[:2]}')
    # print('\n')
    # print(f'bb_labels:\n{bb_labels[:,:2]}')
    # print('\n')

print(f'Idx: {torch.argmin(torch.abs((y[:2] - bb_labels[:,:2])).sum(-1))}')
print('\n')

Idx: 4


Idx: 1


Idx: 4


Idx: 0


Idx: 4




In [ ]:
def calc_targets(bb_preds, labels):
    """Calculate the nearest ground-truth bounding box for each prediciton and use that bb as the target

    Args:
        bb_preds : Predicted boudning boxes     
        labels : Ground-truth labels
    """
    
    
    
    F.l(bb_preds, labels)

In [106]:
X = torch.tensor([[1,2,3,5], [1,3,4,5]], dtype=torch.float)
Y = torch.tensor([[1,2,4,5], [1,1,4,6]], dtype=torch.float)

for x in X:
    print(x)
    minidx = torch.argmin(F.l1_loss(x[2:], Y[:,2:], reduction='none').sum(-1))
    print(minidx)
    print(Y[minidx])
    print('\n')


tensor([1., 2., 3., 5.])
tensor(0)
tensor([1., 2., 4., 5.])


tensor([1., 3., 4., 5.])
tensor(0)
tensor([1., 2., 4., 5.])




C:\Users\rooty\AppData\Local\Temp\ipykernel_13700\2407261444.py:6: UserWarning: Using a target size (torch.Size([2, 2])) that is different to the input size (torch.Size([2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  minidx = torch.argmin(F.l1_loss(x[2:], Y[:,2:], reduction='none').sum(-1))


In [ ]:
def train(model, opt, train, val, epochs=10, pp=100):
    
    model.train()
    # c_loss_fn = nn.CrossEntropyLoss()
    # bb_loss_fn = nn.L1Loss()
    
    for i, data in enumerate(train_loader):

        inputs, labels = data
        
        opt.zero_grad()
        
        bb_pred, class_pred = model(inputs)
                
        targets = calc_targets(bb_preds, labels)
                
        bb_loss = F.l1_loss()
        class_loss = F.cross_entropy()    
        
    

In [2]:
input = torch.randn(3, requires_grad=True)
target = torch.empty(3).random_(2)
loss = F.binary_cross_entropy_with_logits(input, target)
loss.backward()

In [35]:
# Example of target with class indices
loss = nn.CrossEntropyLoss(reduction='none')
# input = torch.randn(3, 5, requires_grad=True)
# target = torch.empty(3, dtype=torch.long).random_(5)
# output = loss(input, target)
# output.backward()
# Example of target with class probabilities
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5).softmax(dim=1)
output = loss(input, target)
# output.backward()

In [30]:
target[:,:]

tensor([[0.7048, 0.0671, 0.1291, 0.0421, 0.0569],
        [0.0366, 0.4363, 0.0321, 0.4402, 0.0547],
        [0.0550, 0.1117, 0.1777, 0.1477, 0.5079]])

In [36]:
print(input)
print(output)

tensor([[-1.4872, -1.8913,  0.9910,  0.2958, -0.0855],
        [ 0.8182,  2.1667, -1.1634, -0.6866, -0.5723],
        [-0.2229, -0.3822,  0.8446,  0.4831, -0.6239]], requires_grad=True)
tensor([2.2302, 3.0668, 1.9288], grad_fn=<NegBackward0>)


In [16]:
torch.log(torch.exp(output * -1) * (torch.exp(input)).sum(-1))

tensor([ 0.0967, -0.6232,  0.1727], grad_fn=<LogBackward0>)